# Introduction to Tutorial 2

In the last tutorial we implemented single-layered neural networks using PyTorch, and we also implemented our own training algorithm.

In this tutorial we would like to experiment by implementing neural networks that have at least one hidden layer, and we would like to start utilising PyTorch's implementation of the backpropogation algorithm to train it rather than using our own .

In particular, we will implement a multi-layer neural network that solves a non-trivial differential equation similar to the kind of equations that arise in many applications such as robotics and autonomous vehicles.

The following is the problem we want to solve using the neural network that we will implement:

하나의 싱글 히든 레이어 뉴럴 네트워크는 트레인 되어진다 / 극점에서 균형을 유지하기 위해서 적절한 힘을 적용하여서.
해석: 적절한 힘을 가해 중력 속에서 막대의 균형을 잡도록 단일 은닉층 신경망을 훈련시키려고 합니다.
의미: 인공지능(신경망)이 막대를 안 쓰러뜨리고 세우는 법을 배우게 하겠다는 목표입니다.


우리 모델에서 우리는 가정한다. 극점 자체는 질량이 없고, 각각 극에 고정된 질량을 가지고 있다. 
해석: 우리 모델에서는(아래 그림 참조), 막대 자체는 질량이 없으며 양 끝에 질량(추)이 고정되어 있다고 가정합니다.

이 pole의 동작은 수직평면에 한정된다. 이 극점의 foot은 오로지 수평으로만 움직인다 (F의 힘을 적용하면서)
해석: 막대의 움직임은 수직 평면 내로 제한됩니다. 막대의 하단부(발)는 힘 $F$를 가함으로써 오직 수평 방향으로만 움직일 수 있습니다.
의미: 막대가 앞뒤로 쓰러지거나 회전하지 않고, 오직 왼쪽/오른쪽으로만 움직이는 2D 환경이라는 뜻입니다.

우리의 유닛, 중력 상수엣 pole의 길이와 질량은 1로 가정된다 
본 모델의 단위계에서 중력 가속도, 막대의 길이, 그리고 질량 값들은 모두 1로 가정합니다.

시간 t의 지점에서 angle 은 수직과 반대 방향인 시계 반대방향으로 측정되며, 이것은 angular velocity 이고, 점점 붙은건은 angluarlar accelation 이다. 
- 막대의 움직임은 식 (1)에 의해 주어집니다. 여기서 시간 $t$에서의 각도 $\phi(t)$는 수직선을 기준으로 시계 반대 방향으로 측정된 것이며, 
- $\dot{\phi}(t)$는 각속도, $\ddot{\phi}(t)$는 각가속도입니다.

등식 2와 같은 힘 F를 적용하는것은 그 극점과 잘 발란스를 맞춘다 특정 범위에서..?ㅋㅋㅋ
해석: 식 (2)에 의해 주어지는 힘 $F$를 가하면, 특정 각도 범위 내에서는 막대의 균형이 꽤 잘 잡힌다는 사실이 밝혀졌습니다.의미: "이렇게 힘을 주면 균형이 잘 잡히더라"라는 일종의 정답(가이드라인)을 미리 제시해 준 것입니다.

<img src="pole.png">

The solve this problem using a multilayer neural network we will utilise the following dataset, which consists of 200 random samples of (angle, angular velocity) pairs, and for each one of these pairs we will also have the value of F that solves equation 1 and 2.

In [106]:
pole_data = [ 
#   , velocity, Force
[0.150833,1.027983,1.779294],
[-1.003966,0.134535,-4.083502],
[-1.004733,1.277974,-2.942121],
[-0.901717,0.066560,-3.855405],
[0.073415,1.333916,1.700663],
[-0.686864,-1.141066,-4.311643],
[-0.982299,-0.835420,-4.994299],
[-1.435255,-1.071030,-6.025172],
[-0.415685,0.499814,-1.519269],
[0.534137,0.549333,3.094825],
[1.451074,-0.662416,4.301793],
[0.011577,-0.418178,-0.360295],
[0.574931,-0.289563,2.429322],
[0.935129,1.518234,5.541612],
[-0.704409,-0.487015,-3.724932],
[-0.863463,1.017197,-2.783291],
[-1.029852,-0.296658,-4.582772],
[-0.419088,-0.463813,-2.498453],
[-0.589456,0.249967,-2.529578],
[1.173519,1.054732,5.665321],
[0.013063,-1.084117,-1.018805],
[1.484438,0.601584,5.582951],
[-0.348765,-1.521637,-3.230324],
[1.101326,0.806323,5.265364],
[0.548997,0.064643,2.673804],
[-0.215117,0.429251,-0.638057],
[0.972999,1.367232,5.500120],
[-1.559747,-0.022842,-5.022537],
[-0.493151,0.946202,-1.420816],
[-0.075429,0.373213,-0.003573],
[-1.111633,0.631928,-3.850189],
[-0.180411,-0.570713,-1.467880],
[-1.235550,0.971273,-3.750372],
[0.536294,-0.254233,2.300536],
[-0.349532,0.138993,-1.573297],
[-0.770322,1.234303,-2.247529],
[0.625648,-0.856656,2.071459],
[0.265067,-1.293889,0.015981],
[-0.807521,-0.204427,-3.817305],
[1.083206,1.312249,5.729571],
[1.430988,-0.702731,4.248483],
[0.170727,0.833215,1.682711],
[-0.906319,0.181753,-3.754443],
[-0.760447,0.171303,-3.274924],
[-0.442865,0.734897,-1.407753],
[-1.026305,0.016275,-4.260680],
[-0.203995,0.364057,-0.648861],
[1.016382,0.131275,4.382320],
[-0.235442,-0.018144,-1.184509],
[1.447814,0.985798,5.948035],
[-1.449971,-0.893280,-5.856828],
[0.649281,0.746497,3.769568],
[-0.179164,-0.656424,-1.547460],
[1.023381,0.584087,4.853450],
[0.709922,0.535767,3.794638],
[0.325563,0.570138,2.169351],
[1.403856,-1.074530,3.855959],
[-0.167468,-1.073283,-1.906713],
[0.677996,0.642858,3.779024],
[0.668840,-1.335642,1.764738],
[-0.193066,1.213307,0.253963],
[0.251405,1.173759,2.417584],
[0.006591,-0.303033,-0.270077],
[-0.265786,1.341922,0.028582],
[1.249595,-0.388792,4.355492],
[0.756900,1.370444,4.803799],
[0.288700,-0.164639,1.258892],
[0.546121,-1.461284,1.135598],
[0.749709,-0.001318,3.405811],
[0.693623,-0.111142,3.085494],
[-1.036372,-0.551634,-4.854443],
[-1.111824,-1.203336,-5.685878],
[-0.055391,0.291480,0.014666],
[-0.705799,-0.948216,-4.191426],
[-0.636482,1.533813,-1.438038],
[-0.713085,0.741224,-2.529628],
[1.176300,1.109140,5.725091],
[0.344163,-0.387929,1.299115],
[-0.764713,-1.492443,-4.954095],
[-0.616828,-1.085891,-3.978144],
[-0.310415,-1.430749,-2.958020],
[-1.286267,1.549057,-3.249912],
[-0.024568,0.830627,0.707801],
[-1.483000,-0.845679,-5.826421],
[-0.741512,0.781395,-2.595623],
[0.613952,-0.207111,2.673399],
[-1.341059,1.072900,-3.795731],
[0.160373,0.174323,0.972754],
[-0.206440,1.025346,0.000461],
[0.796879,0.727850,4.303741],
[0.988339,-1.486979,2.688589],
[-0.101698,0.593818,0.086204],
[1.192934,-1.328355,3.318921],
[-1.364883,-1.142600,-6.036974],
[-1.249979,-0.410939,-5.155828],
[-0.657718,0.010378,-3.046185],
[-0.270867,-0.373213,-1.711050],
[-0.011385,1.275337,1.218413],
[-1.113406,0.076387,-4.409653],
[-1.141162,-0.284146,-4.829736],
[-0.713037,1.043610,-2.227061],
[1.079563,-0.483324,3.925435],
[0.545690,-0.330884,2.264154],
[1.261771,-1.231523,3.531629],
[-0.876358,0.487878,-3.354190],
[1.067099,-1.458840,2.920181],
[0.571671,-0.605419,2.099774],
[0.705751,0.193785,3.436813],
[-0.362954,0.911640,-0.863548],
[0.621957,-0.042161,2.870975],
[-1.070071,1.535059,-2.851115],
[1.538990,0.229833,5.227305],
[-0.408974,-0.043215,-2.031554],
[-0.065650,0.048440,-0.279572],
[-1.537600,0.363961,-4.633284],
[1.335067,-0.679865,4.181856],
[-0.163249,0.843809,0.031184],
[0.407631,-1.188332,0.793848],
[-1.057895,0.098582,-4.258041],
[-0.849082,-0.363482,-4.116854],
[-0.984360,-1.352803,-5.517394],
[-0.251549,1.158084,-0.086438],
[-0.387402,-1.116570,-3.005492],
[-0.218952,0.820416,-0.265617],
[1.365842,-1.167767,3.727585],
[-0.792565,-1.275050,-4.835830],
[-1.203528,-0.824395,-5.490953],
[0.525556,-0.041729,2.466742],
[0.703162,-1.110914,2.122252],
[-1.564109,0.736335,-4.263553],
[0.823868,1.341778,5.010672],
[-1.514303,-0.910202,-5.902225],
[0.614767,0.464101,3.347941],
[-0.527761,1.127692,-1.390311],
[-1.008137,0.193929,-4.035269],
[-0.806610,-0.421677,-4.031406],
[0.411946,0.512661,2.514626],
[-0.834414,-1.546277,-5.250791],
[0.966863,0.517455,4.632997],
[0.844912,0.761885,4.501448],
[0.920460,-1.518473,2.460929],
[1.057656,1.287705,5.643740],
[-0.772096,0.012392,-3.475799],
[-0.324844,1.501839,-0.093968],
[0.472298,-0.318133,1.956538],
[0.667401,-0.274654,2.820081],
[-0.547176,0.723871,-1.877516],
[0.385916,-1.503229,0.378810],
[-0.382848,1.428975,-0.438844],
[1.195235,0.179787,4.831296],
[-1.440576,0.076244,-4.881423],
[-1.043994,1.506872,-2.815223],
[-0.511031,1.138046,-1.307339],
[0.159654,0.145512,0.940395],
[0.462759,-0.541567,1.690525],
[0.923337,0.249392,4.237485],
[0.524166,-0.658485,1.843971],
[1.386838,-0.014357,4.901280],
[-1.291396,0.396366,-4.409739],
[-1.495416,1.467516,-3.518285],
[0.009611,1.087185,1.135241],
[-1.095766,-0.686337,-5.132730],
[0.079791,-0.525364,-0.126833],
[-0.124900,1.360425,0.737550],
[-1.529307,0.317558,-4.678139],
[0.286399,-1.399110,0.013388],
[-1.177019,0.813178,-3.804154],
[-1.463010,-0.117278,-5.088261],
[0.380403,0.267416,2.123891],
[-1.542538,-0.727610,-5.725614],
[1.296621,0.951571,5.764816],
[1.092554,0.249967,4.688993],
[-1.277734,0.908572,-3.878247],
[-1.335162,-0.998310,-5.860143],
[-0.265882,-1.259806,-2.573608],
[-1.101614,1.314502,-3.145190],
[1.398152,-0.626607,4.299062],
[-0.942655,-0.092878,-4.138484],
[0.418801,0.503266,2.536591],
[-0.303273,0.460266,-1.032960],
[-0.749997,1.553947,-1.854235],
[0.631976,-0.356243,2.597459],
[0.796304,0.739738,4.313618],
[1.097252,-0.394065,4.055722],
[-0.563666,1.125486,-1.545957],
[0.449097,-0.837865,1.332895],
[0.506286,-0.029170,2.395491],
[0.982922,0.799324,4.959931],
[-0.691418,1.218532,-1.969618],
[1.371786,0.613472,5.514786],
[1.529499,-1.300600,3.695137],
[0.357154,1.356830,3.104876],
[-0.356435,0.985271,-0.759406],
[-0.306868,-1.508406,-3.018779],
[-0.082284,0.960679,0.549725],
[0.522632,0.738492,3.234305],
[0.943806,-0.416212,3.632772],
[-1.188571,0.169289,-4.469896],
[-1.247294,1.479501,-3.261139],
[1.345996,-0.240188,4.634006]
]

In [113]:
len(pole_data)

200

We will use the dataset to train a NN that takes two inputs (angle, angular velocity) and returns the value of F that solves equation 1 and 2, thus effectively solving the pole balancing differential equation. 

Before implementing the neural network, let's first split the data into training and test (where we use 20% of the data for the later). Again wrapping things in tensors, since we will be using the data as input to a PyTorch NN.

In [107]:
import torch

train_data = pole_data[:160]
test_data = pole_data[160:]


x_train = [row[:2] for row in train_data]
y_train = [row[2] for row in train_data]
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

x_test = [row[:2] for row in test_data]
y_test = [row[2] for row in test_data]
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)


print(y_train.shape)


torch.Size([160])


Now let's implement the template for a NN called PoleNN. The NN contains one hidden layer consisting of three neurons, each which takes the same two inputs. It applies a Sigmoid activation function on the hidden layer, but only a linear activation function on the output layer (why?)

In [ ]:
import torch.nn as nn

class PoleNN(nn.Module):
    def __init__(self): # Initialise (PoleNN)
        super(PoleNN, self).__init__() # Initialise (nn.Module)
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid() ## nn
        
    def forward(self, x):
        # one hidde layer?
        hidden1 = self.fc1(x)
        hidden1 = self.sigmoid(hidden1) # Gap 4 nonlinear function 을 넣어서 곡선, 복잡한 패턴, 비직선적 관계 학습하기 
        output = self.fc3(hidden1)
        return output

# class PoleNN(nn.Module):
#     def __init__(self):
#         super(PoleNN, self).__init__()
#         self.fc1 = nn.Linear(2, 3)
#         self.fc2 = nn.Linear(3, 1)
#         self.fc3 = nn.Linear(4, 1)
#         self.sigmoid = nn.Sigmoid() ## nn
        
#     def forward(self, x):
#         # one hidde layer?
#         hidden1 = self.fc1(x)
#         hidden1 = self.sigmoid(hidden1)
#         hidden2 = self.fc2(hidden1)
#         hidden2 = self.sigmoid(hidden2) # Gap 4 nonlinear function 을 넣어서 곡선, 복잡한 패턴, 비직선적 관계 학습하기 
#         output = self.fc3(hidden2)
#         return output
    
# hidden layer 은 한개이고, 모든 layer 은 3개임 (input, hidden, output)

Now let's get one instance of the above NN, and call it pole_model. Since we didn't specify any intial weights/biases these will all be randomised.

In [96]:
pole_model = PoleNN()

In [97]:
# 이게 왜지??
# model = PoleNN()
# out = model(x_train[:5])

# print(out.shape)   # torch.Size([5, 1])
# print(y_train[:5].shape)

Because we want to train pole_model on our dataset, we need to specify which error loss function we want (in our case it will be sequare error loss - why?) and that we want to optimise using stochastic gradient descent. 

In [ ]:
criterion = torch.nn.MSELoss() # Gap 1
optimizer = torch.optim.SGD(pole_model.parameters(), lr = 0.1)# Gap 2 모델을 어떻게 학습시킬지 결정하는 알고리즘 (loss를 줄이도록 weight를 조금씩 수정하는 역할) (수정할 weights 들, 한번에 얼마나 수정할지)

Notice that when we set up stochastic gradient descent as our optimizer, we specified which parameters we want to optimise, in our case we chose *pole_model.parameters()*, which are all the pole_model parameters, so all weights and biases of all the neuron in the pole_model. We also specified that we want the learning rate to be 0.1.

Now before we train our model, let's first check it's performace to be able to compare how much it improved after we trained it. 

In [116]:
y_pred.shape

torch.Size([40, 1])

In [100]:
y_pred.squeeze().shape

torch.Size([40])

In [101]:
y_test.shape

torch.Size([40])

In [ ]:
pole_model.eval()
y_pred = pole_model(x_test)
before_train = criterion(y_pred.squeeze(), y_test) # loss : y_pred, y_test (예측값, 정답값 ) 
print('Test loss before training' , before_train.item()) 

Test loss before training 12.662588119506836


To evalute the model, we need to get into what is known as evaluation mode, which is done in the first line.

We then obtained the predicted value from the model in the second line, and in the third line we compared the predicted with the actuals using our criteron (which we previously defined as square error loss).

Now let's train our model!

In the following code, we first enter train mode. We then specify that we want to have 20 training epochs.

For each optimise we need to reset the gradient computed in the previous epoch by calling the zero_grad() function. 

The bulk of the training work is done by PyTorch for us in the lines loss.backward() and optimizer.step(), which computes the error loss and then performs backpropogation using our optimization algorithm of choise (in our case we specified it as SGD previously). Finally it also updates the weights of the model accordingly.
우리가 선택한 옵티마이즈 알고리즘을 이용해서 역전 수행한다 

In [ ]:
pole_model.train()
epoch = 20
for epoch in range(epoch):
    optimizer.zero_grad() # opitmizer 을 initalisze하기 위해서 
    y_pred = pole_model(x_train) # Gap 1
    loss = criterion(y_pred.squeeze(), y_train) # Gap 2
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    loss.backward() ## loss 기준으로 gradient 계산 (미분)
    optimizer.step() ## gradient로 weight 업데이트
 
# loss 가 줄어듬 : 모델이 학습하며 똑똑해지는 것 


Epoch 0: train loss: 12.075178146362305
Epoch 1: train loss: 11.931882858276367
Epoch 2: train loss: 11.793124198913574
Epoch 3: train loss: 11.646214485168457
Epoch 4: train loss: 11.482536315917969
Epoch 5: train loss: 11.295675277709961
Epoch 6: train loss: 11.080670356750488
Epoch 7: train loss: 10.833871841430664
Epoch 8: train loss: 10.553098678588867
Epoch 9: train loss: 10.237885475158691
Epoch 10: train loss: 9.889642715454102
Epoch 11: train loss: 9.511630058288574
Epoch 12: train loss: 9.108694076538086
Epoch 13: train loss: 8.686789512634277
Epoch 14: train loss: 8.252426147460938
Epoch 15: train loss: 7.81210994720459
Epoch 16: train loss: 7.371903896331787
Epoch 17: train loss: 6.937135219573975
Epoch 18: train loss: 6.512249946594238
Epoch 19: train loss: 6.100777626037598


Notice in the above that our error improves in each epochs.

There is a marked improvement in the performance of the model on the training data by the end of the 20th epoch. But how well does it generalise? Let's check this by evaluating the trained model on the test dataset we created previously and comparing it's performance:

In [ ]:
# 여기거 5가 나와야 하는데 너무 적은데요 ㅠㅠ
pole_model.eval()
y_pred = pole_model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss after training' , before_train.item())

Test loss before training 6.120377540588379


Great there is also a  marked improvement in the performance of the model on the test dataset! 

Implying that even a simple NN with only one hidden layer is capable of solving sophisticated differential equation, with a relatively small training dataset and a small training run. This highlights the power of multi-layer NN and in particular backpropodation as a means of training them.

# Further exercises:

* In our PoleNN template we only had 3 neurons in our hidden layer. Experiment on the impact on the performance if you increase or decrease the number of neurons in the hidden layer.


* What about when you increase the number of layers?


* Also what is the impact on the performance if you decrease/increase the learning rate?

In [ ]:
# 2는 조금 증가 4 : decrease 

